<a href="https://colab.research.google.com/github/dany444/one/blob/main/Another_One.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.7 MB/s eta 0:00:00


**Для проверки**

In [ ]:
def detect_encoding_manually(byte_data):
    if byte_data.startswith(b'\xef\xbb\xbf'):
        return 'UTF-8 с BOM'
    if byte_data.startswith(b'\xff\xfe'):
        return 'UTF-16 LE'
    if byte_data.startswith(b'\xfe\xff'):
        return 'UTF-16 BE'

    # Проверка на UTF-8 без BOM
    try:
        byte_data.decode('utf-8')
        return 'UTF-8 без BOM'
    except UnicodeDecodeError:
        pass

    # Проверка на ASCII
    if all(byte <= 0x7F for byte in byte_data):
        return 'ASCII'

    # Проверка на Windows-1251
    try:
        byte_data.decode('windows-1251')
        return 'Windows-1251'
    except UnicodeDecodeError:
        pass

    # Проверка на Windows-1252
    try:
        byte_data.decode('windows-1252')
        return 'Windows-1252'
    except UnicodeDecodeError:
        pass

    return 'Неизвестная кодировка'

# Пример использования
byte_data = b'\xef\xbb\xbfHello, World!'  # UTF-8 с BOM
print(detect_encoding_manually(byte_data))  # Выведет: UTF-8 с BOM

byte_data = b'\xff\xfefile content'  # UTF-16 LE
print(detect_encoding_manually(byte_data))  # Выведет: UTF-16 LE

byte_data = b'Hello, World!'  # ASCII
print(detect_encoding_manually(byte_data))  # Выведет: ASCII

byte_data = b'\xd0\x9f\xd0\xb0\xd1\x80\xd0\xb0\xd0\xbc\xd0\xb5\xd1\x82\xd1\x80'  # Windows-1251
print(detect_encoding_manually(byte_data))  # Выведет: Windows-1251

UTF-8 с BOM
UTF-16 LE
UTF-8 без BOM
UTF-8 без BOM


**Сама проверка**

In [ ]:
import os

def test_encoding_detection():
    base_dir = ''
    test_files = {
        #os.path.join(base_dir, "WINDOWS1251_endPDF (1).pdf"): "UTF-8 с BOM",
        #os.path.join(base_dir, "UTF_RusInLaTeX_1.txt"): "UTF-8 без BOM",
        #os.path.join(base_dir, "_UTF-16LE_RusInLaTeX_1.txt"): "UTF-16 LE",
        #os.path.join(base_dir, "UTF_8 with BOM_RusInLaTeX_1.txt"): "UTF-16 BE",
        #os.path.join(base_dir, "converted.pdf"): "Windows-1251",
        #os.path.join(base_dir, "endPDF_Courier (1).pdf"): "Windows-1252",
        os.path.join(base_dir, "endPDF_Courier (1).pdf"): "Windows-1251",
        os.path.join(base_dir, "converted.pdf"): "UTF-8 без BOM",
    }

    for filename, expected in test_files.items():
        with open(filename, 'rb') as f:
            byte_data = f.read()
            detected = detect_encoding_manually(byte_data)
            print(f"Файл: {filename}, Ожидаемая кодировка: {expected}, Обнаруженная кодировка: {detected}")

if __name__ == "__main__":
    test_encoding_detection()

Файл: endPDF_Courier (1).pdf, Ожидаемая кодировка: Windows-1251, Обнаруженная кодировка: Windows-1251
Файл: converted.pdf, Ожидаемая кодировка: UTF-8 без BOM, Обнаруженная кодировка: UTF-8 без BOM


**Перекодировка**

In [ ]:
import fitz  # PyMuPDF
import chardet
import io

def convert_text_from_windows1251_to_utf8(text: bytes) -> str:
    # Проверяем текущую кодировку текста
    detected_encoding = chardet.detect(text)['encoding']
    if detected_encoding == 'windows-1251':
        return text.decode('windows-1251').encode('utf-8').decode('utf-8')
    return text.decode('utf-8')

def convert_pdf(input_pdf_path: str, output_pdf_path: str):
    # Открываем исходный PDF файл
    pdf_document = fitz.open(input_pdf_path)

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text_instances = page.get_text("dict")["blocks"]

        for block in text_instances:
            if block["type"] == 0:  # 0 - текстовый блок
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Конвертируем текст из Windows-1251 в UTF-8
                        span["text"] = convert_text_from_windows1251_to_utf8(span["text"].encode('utf-8'))

    # Сохраняем измененный PDF файл
    pdf_document.save(output_pdf_path)
    pdf_document.close()

if __name__ == "__main__":
    input_pdf_path = "endPDF_Courier (1).pdf"  # Путь к исходному PDF файлу
    output_pdf_path = "converted.pdf"  # Путь к конвертированному PDF файлу

    convert_pdf(input_pdf_path, output_pdf_path)
    print(f"PDF файл успешно конвертирован и сохранен как {output_pdf_path}")

PDF файл успешно конвертирован и сохранен как converted.pdf


In [111]:
#!apt update
#!apt install -y libreoffice
#!pip install pdf2docx python-docx ipywidgets fitz
!pip install frontend

import fitz  # PyMuPDF
import ipywidgets as widgets
from IPython.display import display
import os
import uuid
from pdf2docx import Converter
from docx import Document
import shutil

class PDFEditor:
    def __init__(self, pdf_url=None):
        self.pdf_path = ""
        self.text_blocks = []  # Список текстовых блоков с координатами
        self.text_coords = []  # Список координат для текста
        self.raw_text = ""
        self.temp_pdf_path = None
        self.word_file_path = None
        self.images = []  # Список изображений

        # Загрузка PDF
        self.upload_button = widgets.FileUpload(accept='.pdf', multiple=False)
        self.upload_button.observe(self.load_pdf, names='value')

        # Кнопки
        self.edit_button = widgets.Button(description="Внести правки", disabled=True)
        self.edit_button.on_click(self.extract_text)

        self.save_button = widgets.Button(description="Сохранить изменения", disabled=True)
        self.save_button.on_click(self.save_pdf)

        self.refresh_button = widgets.Button(description="Обновить", disabled=False)
        self.refresh_button.on_click(self.refresh_text)

        # Текстовые поля
        self.encoded_text = widgets.Textarea(layout=widgets.Layout(width='50%', height='500px'))
        self.decoded_text = widgets.HTML(layout=widgets.Layout(width='50%', height='500px'))

        # Поиск
        self.search_input = widgets.Text(placeholder="Введите закодированный текст для поиска", layout=widgets.Layout(width='50%'))
        self.search_button = widgets.Button(description="Найти", disabled=False)
        self.search_button.on_click(self.search_and_highlight)

        # Отображение
        display(self.upload_button, self.edit_button, self.save_button, self.refresh_button)
        display(self.search_input, self.search_button)
        display(widgets.HBox([self.encoded_text, self.decoded_text]))

        if pdf_url:
            self.download_pdf(pdf_url)

    def download_pdf(self, url):
        filename = "/content/final_pdf_with_text (8).pdf"
        os.system(f"wget -O {filename} {url}")
        self.pdf_path = filename
        self.edit_button.disabled = False

    def load_pdf(self, change):
        if self.upload_button.value:
            self.pdf_path = list(self.upload_button.value.keys())[0]
            self.edit_button.disabled = False

    def extract_text(self, event):
        if not self.pdf_path:
            return

        # Извлекаем текст и координаты из оригинального PDF с помощью PyMuPDF
        self.text_blocks = []
        self.text_coords = []
        self.raw_text = ""
        self.images = []

        doc = fitz.open(self.pdf_path)

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]
            for block in blocks:
                if block["type"] == 0:  # Это текстовый блок
                    for line in block["lines"]:
                        for span in line["spans"]:
                            text = span["text"]
                            bbox = span["bbox"]  # Координаты блока текста
                            self.text_blocks.append({"text": text, "bbox": bbox})
                            self.raw_text += text + "\n"

                elif block["type"] == 1:  # Это изображение
                    img = block["image"]
                    image_path = f"/content/image_{uuid.uuid4().hex}.png"
                    with open(image_path, "wb") as img_file:
                        img_file.write(img["image"])

                    # Сохраняем изображения, не как файлы, а как объекты в список
                    self.images.append({"image_path": image_path, "bbox": block["bbox"]})

        # Отображаем извлечённый текст в кодированном и нормальном виде
        self.encoded_text.value = self.encode_text(self.raw_text)
        self.decoded_text.value = self.raw_text

        # Сохраняем путь к временной копии PDF
        self.temp_pdf_path = "/content/temp_copy.pdf"
        shutil.copy(self.pdf_path, self.temp_pdf_path)

        # Разрешаем сохранить файл
        self.save_button.disabled = False

    def encode_text(self, text):
        encoded_text = text.encode('utf-8')
        return ' '.join([f'\\x{b:02x}' for b in encoded_text])

    def search_and_highlight(self, event):
        search_term = self.search_input.value

        if search_term:
            decoded_search_term = self.decode_encoded_text(search_term)
            self.decoded_selected_text = decoded_search_term
            self.update_highlighted_text()

    def decode_encoded_text(self, encoded_text):
        encoded_bytes = bytes([int(b, 16) for b in encoded_text.split("\\x")[1:] if b])
        decoded_text = encoded_bytes.decode('utf-8', errors='ignore')
        return decoded_text

    def update_highlighted_text(self):
        if self.decoded_selected_text:
            highlighted_text = self.highlight_text_in_html(self.raw_text, self.decoded_selected_text)
            self.decoded_text.value = highlighted_text
        else:
            self.decoded_text.value = self.raw_text

    def highlight_text_in_html(self, text, selected_text):
        if selected_text:
            highlighted = f'<span style="background-color: yellow;">{selected_text}</span>'
            return text.replace(selected_text, highlighted)
        return text

    def refresh_text(self, event):
        # Обновление текста в правом окне после изменения в левом
        self.decoded_text.value = self.encoded_text.value

    def save_pdf(self, event):
        if not self.temp_pdf_path:
            return

        # Сохраняем изменения в Word
        doc = Document()

        # Добавляем изменённый текст из закодированного текста
        doc.add_paragraph(self.decode_encoded_text(self.encoded_text.value))

        # Добавляем изображения из оригинального PDF в Word
        for img in self.images:
            image_path = img["image_path"]
            # Мы не вставляем изображение в качестве файла, а просто вставляем его в документ на нужное место
            doc.add_picture(image_path)

        word_temp_path = "/content/updated_word.docx"
        doc.save(word_temp_path)

        # Конвертируем Word обратно в PDF с заданным именем
        new_pdf_path = "/content/EYSPDF_YRA.pdf"
        os.system(f"libreoffice --headless --convert-to pdf {word_temp_path} --outdir /content")

        print(f"Файл сохранён как {new_pdf_path}")


if __name__ == "__main__":
    pdf_url = "/content/final_pdf_with_text (8).pdf"  # Путь к PDF файлу
    app = PDFEditor(pdf_url)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00


FileUpload(value={}, accept='.pdf', description='Upload')

Button(description='Внести правки', disabled=True, style=ButtonStyle())

Button(description='Сохранить изменения', disabled=True, style=ButtonStyle())

Button(description='Обновить', style=ButtonStyle())

Text(value='', layout=Layout(width='50%'), placeholder='Введите закодированный текст для поиска')

Button(description='Найти', style=ButtonStyle())

TypeError: byte indices must be integers or slices, not str